In [2]:
import pandas as pd
import os
from os import path as osp

In [4]:
os.curdir

'.'

In [5]:
ROOT_DIR = '..' # change this
DATA_PATH = osp.join(ROOT_DIR, 'data', 'raw')

print(ROOT_DIR)
print(DATA_PATH)

..
../data/raw


### Read raw data

: 